Implement on Intel® Developer Cloud

# I. Set Up Environment

## Step 1. Init conda

Run this command in console:

```bash
$ conda init
```

NOTE: After this command, you should **restart console manually** to use conda. 

## Step 2. Create a Intel® Extension for PyTorch* environment

Create a new environment with **[Intel® Extension for PyTorch*](https://github.com/intel/intel-extension-for-pytorch)**, run the following commands in console:

```bash
# apply Intel® Extension for PyTorch* for the new environment
$ conda create -n seg --clone pytorch
$ conda activate seg
$ conda install -c anaconda ipykernel
$ python -m ipykernel install --user --name seg
```

NOTE: Please **switch current jupyter kernel to `seg`** for new environment.

## Step 3. Install MMSegmentation

Run the following commands in console:

```bash
# install mim
$ pip install -U openmim 
$ mim install mmengine

# install mmcv
$ git clone https://github.com/open-mmlab/mmcv.git
$ cd mmcv
$ pip install -r requirements/optional.txt
$ pip install -e . -v
$ cd ..

# install mmseg
$ git clone -b main https://github.com/open-mmlab/mmsegmentation.git
$ cd mmsegmentation
$ pip install -v -e .
```

# II. Prepare Dataset

Create a `data` directory and a `cityscapes` directory. Download <u>leftImg8bit_trainvaltest.zip</u> and <u>gtFine_trainvaltest.zip</u> in [Cityscapes](https://www.cityscapes-dataset.com). Unzip files as follows:

Make sure current working directory is `mmsegmentation`, run the following commands in console:

```bash
# create directories
$ mkdir data 
$ cd data && mkdir cityscapes
```
NOTE: Here you should **download [dataset](https://www.cityscapes-dataset.com) to the path: `data/cityscapes`**
```bash
# unzip data files
$ unzip data/cityscapes/leftImg8bit_trainvaltest.zip
$ unzip data/cityscapes/gtFine_trainvaltest.zip

# install cityscapesscripts
$ python -m pip install cityscapesscripts

# convert dataset to mmsegmentation format
$ python tools/dataset_converters/cityscapes.py data/cityscapes --nproc 8
```

# III. Inference

## Step 1. Load a checkpoint

Since a well-performed real-time semantic segmentation model [STDC](https://ieeexplore.ieee.org/document/9577672) has already been trained by OpenMMLab, we can directly use the [checkpoint](https://download.openmmlab.com/mmsegmentation/v0.5/stdc/stdc2_512x1024_80k_cityscapes/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth). Download this checkpoint by `mim download`.

```bash
$ mim download mmsegmentation --config stdc2_4xb12-80k_cityscapes-512x1024 --dest .
$ mkdir ckpt
$ mv stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth ckpt/
```

We can see a checkpoint and its config file:

## Step 2. Evaluate metrics

### 1. Performance Evaluation (mIoU & PA)

#### **1.1 Evaluation on valid set**

In [1]:
!~/.conda/envs/seg/bin/python tools/test.py configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth

10/23 23:33:30 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.16 (main, Jun 15 2023, 02:33:25) [GCC 13.1.0]
    CUDA available: False
    numpy_random_seed: 1319581051
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.0.1a0+cxx11.abi
    PyTorch compiling details: PyTorch built with:
  - GCC 11.2
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.2-Product Build 20230613 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=/opt/rh/gcc-toolset-11/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=1 -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINE

##### **Test time augmentation (TTA)**

Add the following code in `configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py` for valid set:

```python
tta_model = dict(type='SegTTAModel')
tta_pipeline = [
    dict(backend_args=None, type='LoadImageFromFile'),
    dict(
        transforms=[
            [
                dict(keep_ratio=True, scale_factor=0.5, type='Resize'),
                dict(keep_ratio=True, scale_factor=0.75, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.0, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.25, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.5, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.75, type='Resize'),
            ],
            [
                dict(direction='horizontal', prob=0.0, type='RandomFlip'),
                dict(direction='horizontal', prob=1.0, type='RandomFlip'),
            ],
            [
                dict(type='LoadAnnotations'),
            ],
            [
                dict(type='PackSegInputs'),
            ],
        ],
        type='TestTimeAug'),
]
```

Inference with TTA:

In [4]:
!~/.conda/envs/seg/bin/python tools/test.py configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth --tta

10/23 23:51:16 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.16 (main, Jun 15 2023, 02:33:25) [GCC 13.1.0]
    CUDA available: False
    numpy_random_seed: 1209058460
    GCC: gcc (Ubuntu 9.4.0-1ubuntu1~20.04.2) 9.4.0
    PyTorch: 2.0.1a0+cxx11.abi
    PyTorch compiling details: PyTorch built with:
  - GCC 11.2
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2023.2-Product Build 20230613 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - Build settings: BLAS_INFO=mkl, BUILD_TYPE=Release, CXX_COMPILER=/opt/rh/gcc-toolset-11/root/usr/bin/c++, CXX_FLAGS= -D_GLIBCXX_USE_CXX11_ABI=1 -Wno-deprecated -fvisibility-inlines-hidden -DUSE_PTHREADPOOL -DNDEBUG -DUSE_KINE

#### **1.2 Evaluation on test set**

NOTE: Since metric results are given by [Cityscapes](https://www.cityscapes-dataset.com), we only share methods in this section.

Add the following configuration in `configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py`:

```python
test_evaluator = dict(
    type='CityscapesMetric',
    format_only=True,
    keep_results=True,
    output_dir='work_dirs/format_results') # result dir
test_dataloader = dict(
    batch_size=1,
    num_workers=4,
    persistent_workers=True,
    sampler=dict(type='DefaultSampler', shuffle=False),
    dataset=dict(
        type='CityscapesDataset',
        data_root='data/cityscapes/',
        data_prefix=dict(img_path='leftImg8bit/test'),
        pipeline=[
            dict(type='LoadImageFromFile'),
            dict(type='Resize', scale=(2048, 1024), keep_ratio=True),
            dict(type='PackSegInputs')
        ]))
```

Run the following command and submit segmetation results (in `work_dirs/format_results`) to [Cityscapes](https://www.cityscapes-dataset.com):

In [ ]:
!~/.conda/envs/seg/bin/python tools/test.py configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth

##### **Test time augmentation (TTA)**

Add the following code in `configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py` for test set:

```python
tta_model = dict(type='SegTTAModel')
tta_pipeline = [
    dict(backend_args=None, type='LoadImageFromFile'),
    dict(
        transforms=[
            [
                dict(keep_ratio=True, scale_factor=0.5, type='Resize'),
                dict(keep_ratio=True, scale_factor=0.75, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.0, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.25, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.5, type='Resize'),
                dict(keep_ratio=True, scale_factor=1.75, type='Resize'),
            ],
            [
                dict(direction='horizontal', prob=0.0, type='RandomFlip'),
                dict(direction='horizontal', prob=1.0, type='RandomFlip'),
            ],
#             [
#                 dict(type='LoadAnnotations'),
#             ],
            [
                dict(type='PackSegInputs'),
            ],
        ],
        type='TestTimeAug'),
]
```

Inference with TTA:

In [ ]:
!~/.conda/envs/seg/bin/python tools/test.py configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py ckpt/stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth --tta

### 2. Quantization & Deployment

#### 2.1 Prepare the deploy evironment.

Run the following quantization and deployment code based on Intel Openvino.

```bash
$ https://github.com/open-mmlab/mmdeploy.git
$ pip install openvino-dev
$ cd mmdeploy
```

Quantize and deploy the desired model:

```bash
$ ~/.conda/envs/seg/bin/python tools/deploy.py \
            configs/mmseg/segmentation_openvino_static-1024x2048.py \
            stdc2_4xb12-80k_cityscapes-512x1024.py  \
            stdc2_512x1024_80k_cityscapes_20220222_132015-fb1e3a1a.pth \
            demo/resources/cityscapes.png  --work-dir qaunt_models  \
            --device cpu --quant
```

The quantized model would be saved at `qaunt_models`.

#### 2.2 Test the evalution time after quantization and deployment.

In [9]:
!cd ../mmdeploy && ~/.conda/envs/seg/bin/python tools/test.py \
    configs/mmseg/segmentation_openvino_static-1024x2048.py  \
    ../mmsegmentation/configs/stdc/stdc2_4xb12-80k_cityscapes-512x1024.py  \
    --model qaunt_models/end2end.onnx \
    --speed-test 

10/24 02:25:59 - mmengine - WARNING - Failed to search registry with scope "mmseg" in the "Codebases" registry tree. As a workaround, the current "Codebases" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmseg" is a correct scope, or whether the registry is initialized.
10/24 02:25:59 - mmengine - WARNING - Failed to search registry with scope "mmseg" in the "mmseg_tasks" registry tree. As a workaround, the current "mmseg_tasks" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when running the built modules. Please check whether "mmseg" is a correct scope, or whether the registry is initialized.
10/24 02:25:59 - mmengine - WARNING - Failed to search registry with scope "mmseg" in the "backend_segmentors" registry tree. As a workaround, the current "backend_segmentors" registry in "mmdeploy" is used to build instance. This may cause unexpected failure when runni